In [37]:
import h5py
import numpy as np
import os

SAVE_DIR = '/Users/aronjansen/Documents/grainsData/graph_test_data/graphs.hdf5'
file = h5py.File(SAVE_DIR, 'r')

# Structure of data:

- Each timestep has its own entry, i.e. is its own group.
- There is a separate dataset 'contact_params' for the contact parameters
- Also a 'num_steps' just indicating the number of steps
- For each timestep there are the datasets:
    - 'sources'
    - 'destinations'
    - 'node_features'
    - 'edge_features'
    - 'input_features': experimental setup

In [56]:
print(file.keys())
print("contact_params: ", file['contact_params'][:])
print("num_steps: ", np.array(file['num_steps']))

<KeysViewHDF5 ['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '5

# Turn into pytorch geometric graph

In [5]:
import torch
import torch_geometric as tg

/opt/homebrew/Caskroom/miniforge/base/envs/grain-graphs/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
def make_edge_index(data):
    s = torch.tensor(data['sources'][:], dtype=torch.int)
    d = torch.tensor(data['destinations'][:], dtype=torch.int)
    edge_index = torch.stack([s, d])
    
    return edge_index

def make_tg_graph(data):
    edge_index = make_edge_index(data)
    node_features = torch.tensor(data['node_features'][:])
    edge_features = torch.tensor(data['edge_features'][:])
    
    # the node features include position, which we can actually provide separately
    position_idx = 15
    positions = node_features[:, position_idx:position_idx + 3]
    node_features = torch.concat(
        [node_features[:, :position_idx], node_features[:, position_idx + 3:]], axis=1)
    
    graph = tg.data.Data(
        x=node_features,
        edge_index=edge_index,
        edge_attr=edge_features,
        pos=positions,
    )
    return graph


def make_graphs(hdf5_file):
    contact_params = torch.tensor(hdf5_file['contact_params'][:], dtype=torch.float)
    num_steps = int(np.array(hdf5_file['num_steps']))
    
    graphs = [make_tg_graph(hdf5_file[f'{step}']) for step in range(num_steps)]
    for step in range(num_steps - 1):
        graphs[step].y = graphs[step + 1].pos
        
    return graphs

In [66]:
g = make_tg_graph(file['1'])
print(g)
print(g.x)

Data(x=[10000, 18], edge_index=[2, 27353], edge_attr=[27353, 29], pos=[10000, 3])
tensor([[ 6.9998e-01,  1.3319e-07,  1.4410e-08,  ...,  1.7645e-01,
          2.1026e-02,  9.4487e-01],
        [ 6.9994e-01,  6.9876e-08,  3.8542e-07,  ..., -1.0430e-01,
         -3.1843e-02,  9.8436e-01],
        [ 6.9990e-01,  4.9942e-08,  1.8485e-07,  ..., -1.0288e-01,
          6.1068e-02,  9.9264e-01],
        ...,
        [ 3.0014e-01,  2.8647e-07,  9.7508e-08,  ..., -3.4116e-01,
         -3.6463e-01,  7.9001e-01],
        [ 3.0010e-01,  3.0145e-04, -4.2226e-04,  ..., -5.7848e-01,
          5.5913e-02, -7.1321e-01],
        [ 3.0006e-01,  3.6419e-07,  4.4413e-07,  ..., -7.2468e-01,
          1.5819e-01,  6.7068e-01]], dtype=torch.float64)


In [67]:
tst = make_edge_index(file['1'])

In [68]:
print(tg.utils.is_undirected(tst))
bla = torch.tensor([[3, 7], [7, 3]])
print(tg.utils.is_undirected(bla))

False
True


In [69]:
graphs = make_graphs(file)

In [70]:
graphs

[Data(x=[10000, 18], edge_index=[2, 27355], edge_attr=[27355, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 27353], edge_attr=[27353, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 23165], edge_attr=[23165, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 22230], edge_attr=[22230, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 21769], edge_attr=[21769, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 21535], edge_attr=[21535, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 21360], edge_attr=[21360, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 21291], edge_attr=[21291, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 21146], edge_attr=[21146, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18], edge_index=[2, 21095], edge_attr=[21095, 29], pos=[10000, 3], y=[10000, 3]),
 Data(x=[10000, 18],

In [23]:
torch.abs(tst2 - tst2[0]).sum(axis=1).sort()

torch.return_types.sort(
values=tensor([    0,     5,    17,  ..., 11735, 11833, 11882]),
indices=tensor([    0, 16086, 24705,  ..., 17032,  9902,  1516]))

In [15]:
DATA_DIR = '/Users/aronjansen/Documents/grainsData/graph_test_data/'
step = 0
fname = f'simState_drained_200_10000_{step}_gnn_data.hdf5'
f_input = h5py.File(DATA_DIR + fname, 'r')

PRESSURE = '0.1e6'
EXPERIMENT_TYPE = 'drained'
NUM_PARTICLES = '10000'

f_input = f_input[PRESSURE][EXPERIMENT_TYPE][NUM_PARTICLES]

In [16]:
f_input.keys()

<KeysViewHDF5 ['contact_params', 'inputs', 'outputs_bodies', 'outputs_inters']>

In [18]:
f_input['contact_params'][:]

array([[6.96000000e+10, 2.00000000e-01, 2.91456794e-01]])

In [19]:
for key in f_input.keys():
    print(f_input[key])

<HDF5 dataset "contact_params": shape (1, 3), type "<f8">
<HDF5 dataset "inputs": shape (1, 3), type "<f8">
<HDF5 dataset "outputs_bodies": shape (1, 21, 10000), type "<f8">
<HDF5 dataset "outputs_inters": shape (1, 31, 27355), type "<f8">


In [20]:
tst = f_input['outputs_inters']
tst.shape

(1, 31, 27355)

In [21]:
tst[:][0].shape

(31, 27355)

In [23]:
tst[0].shape

(31, 27355)

In [25]:
f_input['inputs'][0]

array([ 5.47688730e-01, -1.00065869e+05,  1.00000000e+04])